In [1]:
import requests
import pandas as pd
import numpy as np
from IPython.display import clear_output
import threading
from threading import Thread
from threading import Lock
from urllib3.exceptions import LocationParseError

lock = Lock()
t_list = []
df = pd.read_csv('malicious_phish.csv')
MURL_NPlist = df.to_numpy()
new_dim = np.zeros((MURL_NPlist.shape[0],3))
pre_shaped_MURL_NPlist = np.append(MURL_NPlist,new_dim,axis=1)

processed_URL = 0
active_url = 0
bengin_url = 0
defacement_url = 0
phishing_url = 0
def crawler_T(start,stop):
    global processed_URL,active_url,bengin_url,defacement_url,phishing_url,pre_shaped_MURL_NPlist
    i = 0
    while i < stop-start+1:
        try:
            try:
                j=0
                while j<3:
                    try:
                        r = requests.get(url=pre_shaped_MURL_NPlist[i+start,0],timeout=5)
                        break
                    except requests.exceptions.Timeout:
                        j+=1
            except requests.exceptions.MissingSchema:
                j=0
                while j<3:
                    try:
                        r = requests.get(url = 'http://' + pre_shaped_MURL_NPlist[i+start,0],timeout=5)
                        break
                    except requests.exceptions.Timeout:
                        j+=1
            # lock.acquire()
            pre_shaped_MURL_NPlist[i+start,2] = r.status_code
            # lock.release()
            # print(r.status_code)
        except:
            i+=1
            # lock.acquire()
            processed_URL+=1
            # lock.release()
            # print(e)
            continue
        if r.status_code == 200:
            # lock.acquire()
            pre_shaped_MURL_NPlist[i+start,3] = 1
            active_url+=1
            pre_shaped_MURL_NPlist[i+start,4] = r.text
            if pre_shaped_MURL_NPlist[i+start,1] == 'benign':
                bengin_url+=1
            elif pre_shaped_MURL_NPlist[i+start,1] == 'defacement':
                defacement_url+=1
            elif pre_shaped_MURL_NPlist[i+start,1] == 'phishing':
                phishing_url+=1
            # lock.release()
        i+=1
        lock.acquire()
        processed_URL+=1
        print('\r', processed_URL, '\nactive:     ', active_url, '\nbenign:     ', bengin_url,'\ndefacement: ', defacement_url, '\nphishing:   ', phishing_url, '\n', threading.activeCount())
        clear_output(wait=True)
        lock.release()

# t1 = Thread(target=crawler_T(0,999))
# t_list.append(t1)
# t1.start()
# t2 = Thread(target=crawler_T(1000,1999))
# t_list.append(t2)
# t2.start()
# t3 = Thread(target=crawler_T(2000,2999))
# t_list.append(t3)
# t3.start()
num_threads = 1000
num_tasks = 40
for i in range(num_threads):
    start = i*num_tasks
    stop = (i + 1)*num_tasks - 1
    t = Thread(target=crawler_T,args=(start,stop))
    t_list.append(t)
    t.start()
for t in t_list:
    t.join()


print(pre_shaped_MURL_NPlist[0:num_threads*num_tasks-1,:])
print('active:     ', active_url)
print('benign:     ', bengin_url)
print('defacement: ', defacement_url)
print('phishing:   ', phishing_url)

[['br-icloud.com.br' 'phishing' 0.0 0.0 0.0]
 ['mp3raid.com/music/krizz_kaliko.html' 'benign' 404 0.0 0.0]
 ['bopsecrets.org/rexroth/cr/1.htm' 'benign' 200 1
  '<html>\n\n<head>\n<title>Classics Revisited (1) (Rexroth)</title>\n<meta name="description" content="Kenneth Rexroth essays on Iliad, Odyssey &amp; Gilgamesh.">\n<meta name="author" content="Kenneth Rexroth">\n<meta name="keywords" content="Homer, Iliad, Odyssey, Gilgamesh, Kenneth Rexroth">\n\n<meta name="Microsoft Border" content="none, default">\n</head>\n\n<body bgcolor="#00FFFF"><table cellSpacing="0" cellPadding="0" width="100%" border="0">\n <tr>\n   <td><p align="center"><font face="Bodoni-DTC" color="#ff0000" size="5">B U R E A U&nbsp;&nbsp; O F&nbsp;&nbsp; P U B L I C&nbsp;&nbsp; S E C R E T S</font></p><hr color="#ff0000"></td>\n </tr>\n</table>\n\n<p>&nbsp;</p>\n\n<h1 align="center">CLASSICS REVISITED (1) </h1>\n\n<p align="center">&nbsp;</p>\n\n<p align="center"><a href="#The Epic of Gilgamesh"><em>The Epic of Gilg

In [2]:
pd.DataFrame(pre_shaped_MURL_NPlist[0:39999,:]).to_csv('40000sample.csv')

In [2]:
import pandas as pd
tdf = pd.read_csv('ProcessedData_html_whois.csv', nrows=10)
print(tdf.columns)


Index(['Unnamed: 0', 'url', 'label', 'response status', 'active bool', 'html',
       '-', '@', '~', '_', '%', '?', '&', '#', 'urlength', 'digitnum',
       'sensitivenum', 'hostlength', 'querylength', 'domain_name',
       'regist_date', 'registrar', 'state'],
      dtype='object')


In [8]:
import pandas as pd

# df = pd.read_csv('40000sample.csv', nrows=1000)
# df = pd.read_csv('40000sample.csv')
df = pd.read_csv('ProcessedData_html_whois.csv')
# print(df)

In [9]:
# df.columns = ['No.','url','label','response_status','active','html']
# print(df)

In [10]:
active_df = df[df['active bool']==1]
active_df.insert(active_df.shape[1],'emptyHTML',False)
active_df.insert(active_df.shape[1],'javascript',False)
# print(ndf)

In [11]:
for each in active_df.index:
    if type(active_df.loc[each]['html']) != type('str'):
        active_df.loc[each,'emptyHTML'] = True
    else:
        if '<script>' in active_df.loc[each]['html']:
            active_df.loc[each,'javascript'] = True


In [12]:
print(active_df[active_df['javascript']==False])

       Unnamed: 0                                                url   
0               0                    bopsecrets.org/rexroth/cr/1.htm  \
1               1  http://buzzfil.net/m/show-art/ils-etaient-loin...   
2               2      espn.go.com/nba/player/_/id/3457/brandon-rush   
10             10                  songfacts.com/detail.php?id=13410   
15             15  http://www.824555.com/app/member/SportOption.p...   
...           ...                                                ...   
17891       17891  http://www.congaltonheating.com/congalton-heat...   
17906       17906  bleacherreport.com/articles/544967-baltimore-r...   
17907       17907  http://www.lgk-luzern.ch/index.php?option=com_...   
17912       17912                         arthur-boyer.blogspot.com/   
17918       17918              maseeh1.tripod.com/advices7/id193.htm   

            label  response status  active bool   
0          benign            200.0          1.0  \
1          benign            200.

In [13]:
from html.parser import HTMLParser

class Myparser(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.scripts_flag = 0
        
        self.hrefs = []
        self.scripts = []
        
        self.hrefs_empty_cnt = 0
        self.hrefs_jsFunc_cnt = 0
        self.hrefs_tel_cnt = 0
        self.hrefs_relative_cnt = 0
        self.hrefs_external_cnt = 0
        # self.hrefs_abnormal_cnt = 0
        
        self.eval_cnt = 0
        self.escape_cnt = 0
        self.exec_cnt = 0
        self.search_cnt = 0
    def handle_starttag(self, tag, attrs):
        if tag == 'a':
            for name, value in attrs:
                if name == 'href':
                    self.hrefs.append(value)
                    # print(value)
                    if value == None or value == '#':
                        self.hrefs_empty_cnt += 1
                    elif value.count('javascript:'):
                        self.hrefs_jsFunc_cnt += 1
                    elif value.count('tel:'):
                        self.hrefs_tel_cnt += 1
                    elif value.count('http://') or value.count('https://'):
                        self.hrefs_external_cnt += 1
                    else:
                        self.hrefs_relative_cnt += 1

        elif tag == 'script':
            self.scripts_flag = 1
    def handle_endtag(self, tag):
        if tag == 'script':
            self.scripts_flag = 0
    def handle_data(self, data):
        if self.scripts_flag == 1:
            self.scripts.append(data)
            # print(type(data))
            self.eval_cnt += data.count('eval(')
            self.escape_cnt += data.count('escape(')
            self.exec_cnt += data.count('exec(')
            self.search_cnt += data.count('search(')
            
           
# myparser = Myparser()
new_feature_list = ['hrefs_empty_cnt',
                    'hrefs_jsFunc_cnt',
                    'hrefs_tel_cnt',
                    'hrefs_relative_cnt',
                    'hrefs_external_cnt',
                    # 'hrefs_abnormal_cnt',
                    'eval_cnt',
                    'escape_cnt',
                    'exec_cnt',
                    'search_cnt']
for each in new_feature_list:
    active_df.insert(active_df.shape[1],each,0)
for each in active_df.index:
    if active_df.loc[each]['emptyHTML'] == False:
        html_text = active_df.loc[each]['html']
        myparser = Myparser()
        myparser.feed(html_text)
        myparser.close()
        active_df.loc[each,'hrefs_empty_cnt'] = myparser.hrefs_empty_cnt
        active_df.loc[each,'hrefs_jsFunc_cnt'] = myparser.hrefs_jsFunc_cnt
        active_df.loc[each,'hrefs_tel_cnt'] = myparser.hrefs_tel_cnt
        active_df.loc[each,'hrefs_relative_cnt'] = myparser.hrefs_tel_cnt
        active_df.loc[each,'hrefs_external_cnt'] = myparser.hrefs_external_cnt
        # active_df.loc[each,'hrefs_abnormal_cnt'] = myparser.hrefs_abnormal_cnt
        active_df.loc[each,'eval_cnt'] = myparser.eval_cnt
        active_df.loc[each,'escape_cnt'] = myparser.escape_cnt
        active_df.loc[each,'exec_cnt'] = myparser.exec_cnt
        active_df.loc[each,'search_cnt'] = myparser.search_cnt
        

In [14]:
print(active_df[active_df['search_cnt']!=0])

       Unnamed: 0                                                url   
5               5      allmusic.com/album/crazy-from-the-heat-r16990  \
71             71  http://mylust.com/videos/232790/hentai-slut-wi...   
130           130                             tomdegan.blogspot.com/   
160           160                      allgame.com/game.php?id=39396   
272           272  http://kakaku.com/kaden/food-processor/ranking...   
...           ...                                                ...   
17705       17705  http://www.ecofutbol.com/index.php?view=articl...   
17715       17715  yellowpages.ca/bus/Quebec/Rosemere/Bleu-Blanc-...   
17774       17774   city-data.com/school/skyline-high-school-ca.html   
17879       17879                              wnd.com/?pageId=27781   
17899       17899                           talkradiousa.tripod.com/   

            label  response status  active bool   
5          benign            200.0          1.0  \
71         benign            200.

In [15]:
active_df.to_csv('active_feature_extracted.csv')

In [16]:
print(active_df.shape)

(17919, 34)
